In [2]:
import cv2

# Extract images from video

In [4]:
def face_detected(image):
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    except:
        print('Image corrupted')
        return False
    face_cascade = cv2.CascadeClassifier('./../detector_architectures/haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 2)
    return len(faces) == 1

def extract_imgs(video_path, dst_path):
    vidcap = cv2.VideoCapture(video_path)
    success, img = vidcap.read()
    count = 0
    while success:
        if face_detected(img):
            cv2.imwrite(dst_path+'/frame%d.jpg'%count, img)
            count += 1
        success, img = vidcap.read()
        
    print("Images extracted to ", dst_path)
    
video_faceA = './videos/Zlatan.mp4'
video_faceB = './videos/Alberto.mp4'


extract_imgs(video_faceA, './dataset/faceA')
extract_imgs(video_faceB, './dataset/faceB')

Images extracted to  ./dataset/faceA
Images extracted to  ./dataset/faceB
